In [1]:
## import libraries
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random


from scipy.ndimage.morphology import binary_erosion as br
from skimage import morphology as skmor

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading'

In [3]:
# fifth (h3k9me3) dataset
## channels: 0 = h3k9me3, 1 =MCM, 2= DAPI
file=f'{bigDir}\\20200604_h3k9me3\\cellinfo_200604v3.csv'
h3k9me3=pd.read_csv(file)

In [4]:
# sixth (h3k9me3) dataset
## channels: 0 = h3k9me3, 1= MCM, 2=DAPI
file=f'{bigDir}\\20200615_h3k9me3\\cellinfo_200615v2.csv'
h3k9me3_2=pd.read_csv(file)

# creating heterochromatin masks - 10% brightest pixel

In [8]:
def create_het_mask(name,data,percentage):
    ## based on the name give as a parameter, selecting the channel 
    ### hp1beta channel is 2 in new dataset and 1 in original dataset
    if name=="new":
        hp1=1
    else:
        hp1=0
    
    
    
    ## calculating 20% brightest pixles
    het_percentile=percentage
    
    # Progress Bar
    i=0
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)

    for i,myCell in islice(data.iterrows(), i,None):

        ## create folder to store this percentile data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}')
        newResultsFolder = os.path.dirname(myCell.path).replace('data_tiff', 'results')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        try:
            os.mkdir(newResultsFolder)
        except:
            pass


        ## open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    

        ## opening the channel according to the dataset
        myChannel = myImage[:, hp1, :, :] 

        ## open nucleus mask
        ### if the file is not found in the directory, trying to change the file extension 
        #### this is because there are some file files that are saved as tiff image 
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
        nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
        try:
            nucleusMask=imread(nucleusMaskPath)
        except:
            nucleusMaskPath=nucleusMaskPath.replace('.tiff','.tif')
            nucleusMask=imread(nucleusMaskPath)


        ## make nucleus mask binary
        nucleusMask = nucleusMask.astype(bool)

        ## clean image
        myNucleus = myChannel[nucleusMask]

        ## find threshold
        myPercentile = np.nanpercentile(myNucleus, (100-het_percentile)) 

        ## threshold
        myHetMask=np.zeros(myChannel.shape).astype('bool')
        myHetMask[myChannel >= myPercentile] = 1

        ## clean from elements outside the nucleus
        cleanNucleusMask=~nucleusMask
        myHetMask[cleanNucleusMask]=0
        cleanHetMask = ~myHetMask

        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_hetChrom.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), myHetMask.astype('uint8'))


        progBar.value=progBar.value+1

        

In [6]:
create_het_mask('h3k9me3',h3k9me3,10)

IntProgress(value=0, description='Progress:', max=78)

In [9]:
create_het_mask('h3k9me3',h3k9me3_2,10)

IntProgress(value=0, description='Progress:', max=88)

In [12]:
create_het_mask('h3k9me3',h3k9me3,50)

IntProgress(value=0, description='Progress:', max=78)

In [13]:
create_het_mask('h3k9me3',h3k9me3_2,50)

IntProgress(value=0, description='Progress:', max=88)

# masking heterochromatin with inner and outer masks

In [5]:
def create_het_mask_erosion (name, data,percentage):
    if name=="new":
        hp1=1
    else:
        hp1=0

    i=0
    het_percentile=percentage
    # Progress Bar
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)

    for i,myCell in islice(data.iterrows(), i,None):

        # create folder to store this percentile (inner and outer) data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}_erosion')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        try:
            os.mkdir(newResultsFolder)
        except:
            pass


        # open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    


        myChannel = myImage[:, 1, :, :] #2nd channel is hetChrom (as stained by HP1B or H3K9Me3)

        ## openning nucleus masks
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','erosion_nuc_mask')
        nucouterpath=nucleusMaskPath.replace('.tif','_eroded_9^5_outer_mask_20.tif')
        nucinnerpath=nucleusMaskPath.replace('.tif','_eroded_9^5_inner_mask_20.tif')
        outernuc=imread(nucouterpath)
        outernuc=outernuc.astype(bool)
        innernuc=imread(nucinnerpath)
        innernuc=innernuc.astype(bool)

        ## opening heterochromatin mask
        hetPath=myCell.path
        hetPath=hetPath.replace('data_tiff',f'segmentation_{percentage}')
        hetPath=hetPath.replace('.tif','_hetChrom.tif')
        het=imread(hetPath)
        het=het.astype(bool)

        ## masking heterochromatin with inner nuc mask
        het_inner=het.copy()
        het_inner=het_inner.astype(bool)
        het_inner[~innernuc]=0

        ## masking heterochromatin with outer nuc mask
        het_outer=het.copy()
        het_outer=het_outer.astype(bool)
        het_outer[~outernuc]=0

        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_hetChrom_inner.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), het_inner.astype('uint8'))
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_hetChrom_outer.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), het_outer.astype('uint8'))


        progBar.value=progBar.value+1

    

In [6]:
create_het_mask_erosion('h3k9me3',h3k9me3_2,10)

IntProgress(value=0, description='Progress:', max=88)

In [7]:
create_het_mask_erosion('h3k9me3',h3k9me3,10)

IntProgress(value=0, description='Progress:', max=78)

In [8]:
create_het_mask_erosion('h3k9me3',h3k9me3_2,50)

IntProgress(value=0, description='Progress:', max=88)

In [9]:
create_het_mask_erosion('h3k9me3',h3k9me3,50)

IntProgress(value=0, description='Progress:', max=78)

# 200402 - recreating euchromatin mask

In [10]:
def create_eu_mask(name,data,percentage):
    ## based on the name give as a parameter, selecting the channel 
    ### hp1beta channel is 2 in new dataset and 1 in original dataset

    if name=="new":
        hp1=1
    else:
        hp1=0

    het_percentile=percentage
    # Progress Bar
    i=0
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)

    for i,myCell in islice(data.iterrows(), i,None):

        ## create folder to store this percentile data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}_eu')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        ## open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    

        ## opening the channel according to the dataset
        myChannel = myImage[:, hp1, :, :] 

        ## open nucleus mask
        ### if the file is not found in the directory, trying to change the file extension 
        #### this is because there are some file files that are saved as tiff image 
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
        nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
        try:
            nucleusMask=imread(nucleusMaskPath)
        except:
            nucleusMaskPath=nucleusMaskPath.replace('.tiff','.tif')
            nucleusMask=imread(nucleusMaskPath)

        hetPath=myCell.path
        hetPath=hetPath.replace('data_tiff',f'segmentation_{percentage}')
        hetPath=hetPath.replace('.tif','_hetChrom.tif')
        het=imread(hetPath)
        het=het.astype(bool)

        ## make nucleus mask binary
        nucleusMask = nucleusMask.astype(bool)

        ## clean image
        myNucleus = myChannel[nucleusMask]

        ## masking heterochromatin pixels out of nucleus - so total nucleus - heterochromatin = euchromatin
        myEuMask=np.zeros(myChannel.shape).astype('bool')
        myEuMask[nucleusMask]=1
        myEuMask[het]=0
        
        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_euChrom.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), myEuMask.astype('uint8'))
        

        progBar.value=progBar.value+1
        

In [11]:
create_eu_mask('h3k9me3',h3k9me3_2,10)

IntProgress(value=0, description='Progress:', max=88)

In [12]:
create_eu_mask('h3k9me3',h3k9me3,10)

IntProgress(value=0, description='Progress:', max=78)

In [13]:
create_eu_mask('h3k9me3',h3k9me3,50)

IntProgress(value=0, description='Progress:', max=78)

In [14]:
create_eu_mask('h3k9me3',h3k9me3_2,50)

IntProgress(value=0, description='Progress:', max=88)

# creating euchromatin inner and outer mask 

In [15]:
def create_eu_mask_erosion (name, data,percentage):
    if name=="new":
        hp1=1
    else:
        hp1=0

    i=0
    
    het_percentile=percentage
    eu_percentile=100-percentage
    
    # Progress Bar
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)

    for i,myCell in islice(data.iterrows(), i,None):

        # create folder to store this percentile (inner and outer) data if it doesn't exist yet
        newSegFolder = os.path.dirname(myCell.path).replace('data_tiff',f'segmentation_{het_percentile}_eu_erosion')

        try:
            os.mkdir(newSegFolder)
        except:
            pass

        try:
            os.mkdir(newResultsFolder)
        except:
            pass


        # open tiff image of a cell
        try:
            myImage=imread(myCell.path)    
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)    


        myChannel = myImage[:, 1, :, :] #2nd channel is hetChrom (as stained by HP1B or H3K9Me3)

        ## openning nucleus masks
        nucleusMaskPath=myCell.path
        nucleusMaskPath=nucleusMaskPath.replace('data_tiff','erosion_nuc_mask')
        nucouterpath=nucleusMaskPath.replace('.tif','_eroded_9^5_outer_mask_20.tif')
        nucinnerpath=nucleusMaskPath.replace('.tif','_eroded_9^5_inner_mask_20.tif')
        outernuc=imread(nucouterpath)
        outernuc=outernuc.astype(bool)
        innernuc=imread(nucinnerpath)
        innernuc=innernuc.astype(bool)

        ## opening heterochromatin mask
        euPath=myCell.path
        euPath=euPath.replace('data_tiff',f'segmentation_{percentage}_eu')
        euPath=euPath.replace('.tif','_euChrom.tif')
        eu=imread(euPath)
        eu=eu.astype(bool)

        ## masking heterochromatin with inner nuc mask
        eu_inner=eu.copy()
        eu_inner=eu_inner.astype(bool)
        eu_inner[~innernuc]=0

        ## masking heterochromatin with outer nuc mask
        eu_outer=eu.copy()
        eu_outer=eu_outer.astype(bool)
        eu_outer[~outernuc]=0

        ## save segmentation mask
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_euChrom_inner.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), eu_inner.astype('uint8'))
        maskFile = os.path.basename(myCell.path).replace('.tif',f'_euChrom_outer.tif')
        tifffile.imsave(os.path.join(newSegFolder,maskFile), eu_outer.astype('uint8'))


        progBar.value=progBar.value+1

    

In [16]:
create_eu_mask_erosion('h3k9me3',h3k9me3_2,10)

IntProgress(value=0, description='Progress:', max=88)

In [17]:
create_eu_mask_erosion("h3k9me3",h3k9me3,10)

IntProgress(value=0, description='Progress:', max=78)

In [18]:
create_eu_mask_erosion('h3k9me3',h3k9me3_2,50)

IntProgress(value=0, description='Progress:', max=88)

In [19]:
create_eu_mask_erosion("h3k9me3",h3k9me3,50)

IntProgress(value=0, description='Progress:', max=78)